In [1]:
import pandas as pd

In [5]:
data_train = pd.read_csv('../../data/credit_scoring_train.csv',
                        index_col='client_id')
data_test = pd.read_csv('../../data/credit_scoring_test.csv',
                       index_col='client_id')

In [3]:
data_train.shape

(75000, 11)

In [6]:
data_train.head()

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit,Delinquent90
client_id,,,,,,,,,,
0,0.496289,49.1,13,0,0.0,2,0,5298.360639,0.387028,0
1,0.433567,48.0,9,2,2.0,1,0,6008.056256,0.234679,0
2,2206.731199,55.5,21,1,NaN,1,0,NaN,0.348227,0
3,886.132793,55.3,3,0,0.0,0,0,NaN,0.971930,0
4,0.000000,52.3,1,0,0.0,0,0,2504.613105,1.004350,0


In [7]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 0 to 74999
Data columns (total 10 columns):
DIR                      75000 non-null float64
Age                      75000 non-null float64
NumLoans                 75000 non-null int64
NumRealEstateLoans       75000 non-null int64
NumDependents            73084 non-null float64
Num30-59Delinquencies    75000 non-null int64
Num60-89Delinquencies    75000 non-null int64
Income                   60153 non-null float64
BalanceToCreditLimit     75000 non-null float64
Delinquent90             75000 non-null int64
dtypes: float64(5), int64(5)
memory usage: 6.3 MB


In [8]:
data_train.dropna().shape

(60153, 10)

In [9]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 75000 to 149999
Data columns (total 9 columns):
DIR                      75000 non-null float64
Age                      75000 non-null float64
NumLoans                 75000 non-null int64
NumRealEstateLoans       75000 non-null int64
NumDependents            72992 non-null float64
Num30-59Delinquencies    75000 non-null int64
Num60-89Delinquencies    75000 non-null int64
Income                   60116 non-null float64
BalanceToCreditLimit     75000 non-null float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


In [10]:
data_train['Income'].fillna(data_train['Income'].median(), inplace=True)
data_test['Income'].fillna(data_train['Income'].median(), inplace=True)
data_train['NumDependents'].fillna(data_train['NumDependents'].median(), 
                                   inplace=True)
data_test['NumDependents'].fillna(data_train['NumDependents'].median(), 
                                  inplace=True)

In [12]:
X_train = data_train.drop('Delinquent90', axis=1).values
y_train = data_train['Delinquent90'].values
X_test = data_test.values

In [17]:
import numpy as np

In [18]:
np.bincount(y_train)

array([69987,  5013])

In [13]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [23]:
X_train_part, X_ho_part, y_train_part, y_ho_part  = \
        train_test_split(X_train, y_train,  test_size=0.3, 
                        random_state=17)

In [19]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [16]:
from sklearn.metrics import roc_auc_score

In [24]:
logit = LogisticRegression(random_state=17, n_jobs=-1)
logit.fit(X_train_part, y_train_part)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
first_logit_pred = logit.predict_proba(X_ho_part)[:, 1]

In [28]:
roc_auc_score(y_ho_part, first_logit_pred)

0.69016727228154806

In [29]:
scaler = StandardScaler().fit(X_train_part, y_train_part)

In [30]:
X_train_part_scaled = scaler.transform(X_train_part)
X_train_ho_scaled = scaler.transform(X_ho_part)

In [31]:
logit2 = LogisticRegression(random_state=17, n_jobs=-1)
logit2.fit(X_train_part_scaled, y_train_part)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [32]:
second_logit_pred = logit2.predict_proba(X_train_ho_scaled)[:, 1]

In [33]:
roc_auc_score(y_ho_part, second_logit_pred)

0.68818098025693086

In [36]:
gbm = GradientBoostingClassifier(random_state=17).fit(X_train_part, y_train_part)

In [39]:
GradientBoostingClassifier?

In [37]:
first_gbm_pred = gbm.predict_proba(X_ho_part)[:, 1]

In [38]:
roc_auc_score(y_ho_part, first_gbm_pred)

0.84142553215664306

In [40]:
from sklearn.model_selection import RandomizedSearchCV

In [42]:
RandomizedSearchCV?

In [51]:
params = {'max_depth': range(2, 7),
         'min_samples_leaf': range(10, 14)}

gbm_grid = RandomizedSearchCV(estimator=gbm, param_distributions=params, 
                              n_iter=20, scoring='roc_auc',
                             n_jobs=-1, verbose=True)

In [52]:
%%time
gbm_grid.fit(X_train_part, y_train_part)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   53.5s finished


CPU times: user 5.01 s, sys: 107 ms, total: 5.12 s
Wall time: 58.1 s


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=17,
              subsample=1.0, verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=20, n_jobs=-1,
          param_distributions={'max_depth': range(2, 7), 'min_samples_leaf': range(10, 14)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='roc_auc', verbose=True)

In [53]:
gbm_grid.best_score_, gbm_grid.best_params_

(0.84615311413426397, {'max_depth': 4, 'min_samples_leaf': 12})

In [55]:
roc_auc_score(y_ho_part, gbm_grid.predict_proba(X_ho_part)[:, 1])

0.84413089958044596

In [56]:
from sklearn.preprocessing import PolynomialFeatures

In [66]:
logit_pipe = Pipeline([('scaler', StandardScaler()),
                        ('poly', PolynomialFeatures(degree=2)),
                       ('logit', LogisticRegression(random_state=17,
                                                   n_jobs=-1))])

In [75]:
logit_pipe2 = Pipeline([('scaler', StandardScaler()),
                       ('logit', LogisticRegression(random_state=17,
                                                   n_jobs=-1))])

In [69]:
logit_params = {'logit__C': np.logspace(-2, 2, 5)}


logit_grid = GridSearchCV(logit_pipe, logit_params, n_jobs=-1,
                         verbose=True)

In [70]:
logit_grid.fit(X_train_part, y_train_part)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   28.8s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('poly', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('logit', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'logit__C': array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [72]:
logit_grid.best_score_, logit_grid.best_params_

(0.93398095238095236, {'logit__C': 0.10000000000000001})

In [76]:
logit_params = {'logit__C': np.logspace(-2, 2, 5)}


logit_grid2 = GridSearchCV(logit_pipe2, logit_params, n_jobs=-1,
                         verbose=True).fit(X_train_part, y_train_part)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.6s finished


In [77]:
logit_grid2.best_score_

0.93350476190476195

In [78]:
roc_auc_score(y_ho_part, logit_grid2.predict_proba(X_ho_part)[:, 1])

0.68673328268640277

In [74]:
roc_auc_score(y_ho_part, logit_grid.predict_proba(X_ho_part)[:, 1])

0.75755133244612005

In [79]:
gbm_grid.best_estimator_.feature_importances_

array([ 0.1191496 ,  0.11119122,  0.07466021,  0.05094255,  0.0189772 ,
        0.1048229 ,  0.09578791,  0.11739339,  0.30707502])

In [81]:
pd.DataFrame({'feat': data_train.columns[:-1], 
             'importance': gbm_grid.best_estimator_.feature_importances_})

,feat,importance
0,DIR,0.119150
1,Age,0.111191
2,NumLoans,0.074660
3,NumRealEstateLoans,0.050943
4,NumDependents,0.018977
5,Num30-59Delinquencies,0.104823
6,Num60-89Delinquencies,0.095788
7,Income,0.117393
8,BalanceToCreditLimit,0.307075


In [84]:
logit = LogisticRegression(C=0.1, n_jobs=-1, random_state=17)

In [85]:
logit.fit(X_train_part_scaled, y_train_part)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [89]:
logit.coef_

array([[-0.04097316, -0.40814799, -0.03949293,  0.04470868,  0.12580228,
         2.09589141, -1.95245514, -0.62431705,  0.01297359]])

In [90]:
pd.DataFrame({'feat': data_train.columns[:-1], 
             'coef': logit.coef_.flatten()}).sort_values(by='coef')

,coef,feat
6,-1.952455,Num60-89Delinquencies
7,-0.624317,Income
1,-0.408148,Age
0,-0.040973,DIR
2,-0.039493,NumLoans
8,0.012974,BalanceToCreditLimit
3,0.044709,NumRealEstateLoans
4,0.125802,NumDependents
5,2.095891,Num30-59Delinquencies


In [57]:
a = gbm_grid.predict_proba(X_ho_part)[:10]

In [58]:
a.shape

(10, 2)

In [59]:
a

array([[ 0.99048819,  0.00951181],
       [ 0.95286231,  0.04713769],
       [ 0.98873847,  0.01126153],
       [ 0.98343463,  0.01656537],
       [ 0.88681351,  0.11318649],
       [ 0.90809999,  0.09190001],
       [ 0.95501399,  0.04498601],
       [ 0.57846685,  0.42153315],
       [ 0.60050932,  0.39949068],
       [ 0.98725058,  0.01274942]])

In [60]:
poly = PolynomialFeatures(degree=2)

In [61]:
poly.fit(a)

PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)